In [1]:
import json
with open('router_data/1128_newcon2_2.json','r') as f:
    data = json.load(f)

In [2]:
# 收集成功的证明
success = []
for item in data:
    if item['success']['1128_newcon2_2']:
        success.append(item)

In [3]:
# 进行ij反解
news = set()
for item in success:
    name = item['name']
    try:
        i = int(name.split('_')[-2])
        j = int(name.split('_')[-1])
        item['id'] = (i,j)
        item['kname'] = '_'.join(name.split('_')[:-2])
    except:
        item['id'] = (-1,-1)
        item['kname'] = name
    news.add(item['kname'])


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5"
from transformers import AutoTokenizer, AutoModel
import torch
# 选择预训练模型，例如BERT
model_name = ['deepseek-ai/DeepSeek-Prover-V1.5-RL','/home/limengze/DeepSeek-Coder/finetune/finetune_result5/checkpoint-40']
tokenizer = AutoTokenizer.from_pretrained(model_name[0])
model = AutoModel.from_pretrained(model_name[0],device_map='auto')
model.config.output_hidden_states = True

/home/limengze/miniconda3/envs/ATP_Train/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/limengze/miniconda3/envs/ATP_Train/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/limengze/miniconda3/envs/ATP_Train/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/limengze/miniconda3/envs/ATP_Train/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.uti

In [4]:
with open('1127con/aime_amc_new_con2.jsonl','r') as f:
    raw = f.readlines()
    raw = [json.loads(item) for item in raw]
# 读取原始数据

In [5]:
# 拿到所有subgoals的嵌入
with open('1127con/seg_map.json','r') as f:
    seg_map = json.load(f)
from tqdm import tqdm
with torch.no_grad():
    for item in tqdm(seg_map):
        emb_list=[]
        if type(seg_map[item])== str:
            tokens=tokenizer(text, return_tensors='pt')
            output = model(**tokens)
            layers = []
            # 保存输出
            for i in range(len(output.hidden_states)):
                embeddings = output.hidden_states[i]
                embeddings = embeddings.numpy().mean(axis=1)
                layers.append(embeddings)
            emb_list.append(layers)
            text0 = seg_map[item]
        else:
            for text in seg_map[item]:
                tokens=tokenizer(text, return_tensors='pt')
                output = model(**tokens)
                layers = []
                # 保存输出
                for i in range(len(output.hidden_states)):
                    embeddings = output.hidden_states[i]
                    embeddings = embeddings.numpy().mean(axis=1)
                    layers.append(embeddings)
                emb_list.append(layers)
            text0 = seg_map[item].copy()
        seg_map[item] = {'emb':emb_list,'text':text0}


NameError: name 'torch' is not defined

"Certainly! Let's fill in the details of each step in the outlined proof.\n\n**Problem Explanation:**\nSusie pays for \\(4\\) muffins and \\(3\\) bananas, and Calvin spends twice as much on \\(2\\) muffins and \\(16\\) bananas. We need to determine how many times more expensive a muffin is compared to a banana.\n\n**Define Variables:**\nLet \\( m \\) be the cost of one muffin and \\( b \\) be the cost of one banana.\n\n**1. Susie's and Calvin's Total Costs:**\n- Susie's cost: \\( 4m + 3b \\)\n- Calvin's cost: \\( 2m + 16b \\)\n- Given: \\( \\text{Calvin's cost} = 2 \\times \\text{Susie's cost} \\)\n\nThis leads to the equation:\n\\[ 2m + 16b = 2(4m + 3b) \\]\n\n**2. Simplification and Rearrangement:**\nSimplify the equation:\n\\[ 2m + 16b = 8m + 6b \\]\n\nRearrange by moving all terms involving \\( m \\) to one side and terms involving \\( b \\) to the other side:\n\\[ 2m - 8m + 16b - 6b = 0 \\]\n\nCombine like terms:\n\\[ -6m + 10b = 0 \\]\n\n**3. Express \\( m \\) in terms of \\( b \

In [6]:
import pickle
# with open('1127con/seg_map_emb.pkl','wb') as f:
#     pickle.dump(seg_map,f)
# 读取权重
with open('1127con/seg_map_emb.pkl','rb') as f:
    seg_map = pickle.load(f)

In [89]:
# 在news阶段就要划分数据集
import random
random.seed(0)
news = list(news)
random.shuffle(news)
# 训练集
train = news[:int(len(news)*0.5)]
# 测试集
test = news[int(len(news)*0.5):]

In [ ]:
# 构建良性数据集
good_vectors = {}
bad_vectors = {}
for item in success:
    key = item['kname']
    if item['id'] == (-1,-1):
        continue # 不与采用
        print(len(seg_map[item['name']]['emb']))
    else:
        i,j = item['id']
        if key in good_vectors.keys():
            good_vectors[key].add(i)
            good_vectors[key].add(j)
        else:
            good_vectors[key] = set([i,j])

for item in list(news):
    d=len(seg_map[item]['emb'])
    bad_vectors[item] = set(range(d))
for item in good_vectors:
    bad_vectors[item] = bad_vectors[item] - good_vectors[item]

# 清除bad_vectors中的空集
bad_vectors = {k:v for k,v in bad_vectors.items() if len(v)>0}

In [ ]:
# mark 中的向量都是训练集的
mark=[]
for item in train:
    if item in good_vectors.keys():
        for i in good_vectors[item]:
            mark.append((seg_map[item]['emb'][i],1))
for item in train:
    if item in bad_vectors.keys():
        for i in bad_vectors[item]:
            mark.append((seg_map[item]['emb'][i],-1))



In [127]:
# 定义最近邻分类
def dist(x,y):
    layer = 2
    vec1 = x[layer]
    vec2 = y[layer]
    return ((vec1-vec2)**2).sum()**0.5


def classify(x,Y):
    dists =  [dist(x,y[0]) for y in Y]
    one=0
    for i in range(len(dists)):
        one += Y[i][1]/dists[i]
    one = one/len(dists)
    return one

def Test(emb,ans,Y):
    ones = [(classify(emb[i],Y),i) for i in range(len(emb))]
    # 对ones进行排序,根据key
    ones = sorted(ones,key=lambda x:-x[0])
    print(ones)
    print(ans)
    if ones[0][1] in ans and ones[1][1] in ans:
        return 1
    else:
        return 0

In [ ]:
# seg_map[test[0]] embeding
#  good_vectors[test[0]] answer


{0, 2, 4, 5}

In [128]:
# 统计正确率
total = 0
correct = 0
for item in test:
    try:
        correct+=Test(seg_map[item]['emb'],good_vectors[item],mark)
        total+=1
    except:
        pass
print(correct/total)

[(np.float32(0.023542628), 4), (np.float32(0.021882748), 0), (np.float32(0.02145388), 5), (np.float32(0.0164782), 2), (np.float32(0.015883036), 1), (np.float32(0.015849581), 3), (np.float32(0.014806216), 6)]
{0, 2, 4, 5}
[(np.float32(0.039501518), 2), (np.float32(0.030569986), 1), (np.float32(0.019862587), 0), (np.float32(0.015402619), 3), (np.float32(0.014718563), 4)]
{1, 2, 3, 4}
[(np.float32(0.03385408), 2), (np.float32(0.032268986), 6), (np.float32(0.031620406), 0), (np.float32(0.02542017), 4), (np.float32(0.018254077), 5), (np.float32(0.017550088), 3), (np.float32(0.011805613), 1)]
{0, 1, 2, 3, 4, 5, 6}
[(np.float32(0.0363795), 2), (np.float32(0.02066771), 1), (np.float32(0.009717819), 0)]
{0, 1, 2}
[(np.float32(0.022958659), 3), (np.float32(0.015790846), 2), (np.float32(0.015344508), 1), (np.float32(0.0142891295), 0)]
{0, 1}
[(np.float32(0.030891642), 6), (np.float32(0.030462552), 5), (np.float32(0.026765745), 4), (np.float32(0.026349768), 1), (np.float32(0.025482452), 2), (np.fl